In [12]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [13]:
data = pd.read_csv("initial_data.csv")

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min

print(y_range)

NOx                  685.846823
PM 1                   5.547569
CO2                  160.188892
PM 2                  15.561156
Pressure cylinder    115.101073
dtype: float64


In [14]:
# Datenaufteilung in Training und Test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)



# Umwandlung in PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [15]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 16)
        self.layer2 = nn.Linear(16, 12)
        self.layer3 = nn.Linear(12, 10)
        self.output = nn.Linear(10, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [16]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 300
batch_size = 3

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [17]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

Epoch [10/300], Loss: 0.1751
Epoch [20/300], Loss: 0.1217
Epoch [30/300], Loss: 0.0807
Epoch [40/300], Loss: 0.1748
Epoch [50/300], Loss: 0.0815
Epoch [60/300], Loss: 0.1586
Epoch [70/300], Loss: 0.2009
Epoch [80/300], Loss: 0.1073
Epoch [90/300], Loss: 0.0765
Epoch [100/300], Loss: 0.0515
Epoch [110/300], Loss: 0.7049
Epoch [120/300], Loss: 0.2243
Epoch [130/300], Loss: 0.0824
Epoch [140/300], Loss: 0.0865
Epoch [150/300], Loss: 0.0916
Epoch [160/300], Loss: 0.0222
Epoch [170/300], Loss: 0.0860
Epoch [180/300], Loss: 0.1912
Epoch [190/300], Loss: 0.0510
Epoch [200/300], Loss: 0.0590
Epoch [210/300], Loss: 0.1619
Epoch [220/300], Loss: 0.0566
Epoch [230/300], Loss: 0.0774
Epoch [240/300], Loss: 0.0929
Epoch [250/300], Loss: 0.0625
Epoch [260/300], Loss: 0.3708
Epoch [270/300], Loss: 0.0293
Epoch [280/300], Loss: 0.0957
Epoch [290/300], Loss: 0.0489
Epoch [300/300], Loss: 0.0300


In [18]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    #print(y_test)
    
    print(f'Test Loss: {test_loss.item():.4f}')
    y_pred = scaler_y.inverse_transform(y_pred)
    y_test = scaler_y.inverse_transform(y_test)
    for pred, test in zip(y_pred, y_test):
        print(str(pred) + ":" + str(test))

Test Loss: 0.3438
[41.14299944  0.58214004 21.7872745   2.15640972 55.62369427]:[45.11063706  0.37043239 13.33908759  1.34438471 50.75279046]
[131.71369706   3.13554281  62.38156442  10.72217058  70.83469976]:[144.31417996   3.2442979   63.07336518   9.38207516  71.47676054]
[65.97137429  1.93152984 36.64003307  7.52564334 59.87109941]:[66.86942417  4.09588526 36.29146793 16.28325513 60.49792953]
[67.11721887  2.51848961 38.3940865  10.73083023 60.01219377]:[94.23299035  2.83292106 38.50904745 10.53630092 59.95670593]
[49.19917043  0.93842533 20.32735261  4.85778913 56.91254837]:[41.90699086  0.76024322 22.40993205  4.24715129 54.66871374]
[89.43849394  2.96160777 48.77547089 12.15301307 63.13492179]:[84.29989805  3.41757813 48.34347086 14.36933964 66.03415315]
[563.61712844   1.03111941  78.90135886   2.3484306  110.40902448]:[465.21158656   0.99647182  76.01429871   2.44630606 107.39016145]
[195.97349465   3.43658363 101.34686009   6.28731369 102.67766736]:[165.12383548   3.63098576 

In [19]:
# build matrix of elementwise mean squared error

from sklearn.metrics import mean_squared_error


matrix = []
for pred, test in zip(y_pred, y_test):
    data_set_mse = []
    for p, t in zip(pred, test):
        #print(str(pred) + ":" + str(test))
        mse = mean_squared_error([p], [t])
        data_set_mse.append(mse)
    #print(data_set_mse)

    percentage_to_range = data_set_mse / y_max
    matrix.append(percentage_to_range)
#print(matrix)


In [20]:
# check mse per dataset row

from statistics import mean 

mean_for_predicted_dataset = []

for predicted_dataset in matrix:
    mean_for_predicted_dataset.append(mean(predicted_dataset))

print("\n")
for entry in mean_for_predicted_dataset:
    print(entry)

print("Mean:")
print(mean(mean_for_predicted_dataset))



0.13132318641588256
0.0686918512722262
1.1072926366140783
0.21288881449044245
0.03241534084134923
0.08586592323399748
2.7734928963021312
0.31359451587335774
0.08051339105120783
0.12970280456950534
Mean:
0.4935781360664178


In [23]:
# Ceck Accuracy among features of test data

#print(y_pred.shape)

y_pred_t = np.transpose(y_pred)
y_test_t = np.transpose(y_test)

# print(y_pred_t.shape)
# print(y_test_t.shape)
featurewise_mean = []
for feature_t, feature_p in zip(y_test_t, y_pred_t):
    # print(feature_t)
    # print(feature_p)
    # print(mean_squared_error(feature_t, feature_p))
    featurewise_mean.append(mean_squared_error(feature_t, feature_p))

print(featurewise_mean)

print(y_range)
featurewise_mean_percentage = featurewise_mean / y_max
print("mse per feature in percentage to value range")
print(featurewise_mean_percentage)

[1197.847338205802, 0.5148077444893129, 10.983256378589118, 9.203454804332717, 6.8953256634269335]
NOx                  685.846823
PM 1                   5.547569
CO2                  160.188892
PM 2                  15.561156
Pressure cylinder    115.101073
dtype: float64
mse per feature in percentage to value range
NOx                  1.701912
PM 1                 0.090338
CO2                  0.067349
PM 2                 0.565210
Pressure cylinder    0.043083
dtype: float64
